<a href="https://colab.research.google.com/github/Kyungbok-ai-study/ntl/blob/main/%EC%A0%9C48%ED%9A%8C_%EB%AC%BC%EB%A6%AC%EC%B9%98%EB%A3%8C%EC%82%AC_%EB%AC%B8%EC%A0%9C%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 파일 업로드 및 환경 설정

Google Colab에 서비스 계정 키(`.json`)와 OCR할 PDF 파일을 업로드합니다.  
그 다음, 필요한 패키지를 설치합니다.

- `.json`: Google Cloud Vision API 키 파일
- `.pdf`: 스캔된 시험지 원본 파일
- `poppler-utils`: PDF → 이미지 변환을 위한 도구
- `google-cloud-vision`, `pdf2image`, `google-cloud-storage`: OCR 및 스토리지 연동용 라이브러리

🔧 실행 후 필요한 모든 파일과 도구가 준비됩니다.


In [1]:
from google.colab import files
uploaded = files.upload()  #컴퓨터에서 .json 파일을 선택하세요


Saving ntlkbuai-9302f377d55f.json to ntlkbuai-9302f377d55f.json
Saving 2020년도 제48회 물리치료사 국가시험_1교시(홀수형).pdf to 2020년도 제48회 물리치료사 국가시험_1교시(홀수형).pdf


In [3]:
!apt-get install -y poppler-utils
!pip install --upgrade google-cloud-vision pdf2image google-cloud-storage

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 0 B/186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Selecting previously unselected package poppler-utils.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


## 2. PDF 페이지 분할 및 Google Vision OCR 실행

이 단계에서는 스캔된 PDF 파일을 이미지로 변환하고, 각 페이지에서 텍스트를 추출(OCR)합니다.

- 첫 번째 페이지는 안내문이므로 전체 OCR 수행
- 2페이지부터는 2단 레이아웃을 고려하여 좌/우로 나누어 OCR 수행
- 상단(Header)과 하단(Footer)은 비율 기준으로 제거하여 노이즈를 줄임
- OCR 결과는 `output/page_번호.txt` 형태로 저장

📌 사용 기술:
- `pdf2image` : PDF → 이미지 변환
- `OpenCV` : 이미지 자르기 및 저장
- `Google Cloud Vision API` : OCR (문서 텍스트 인식)

```python
run_ocr()           # Vision API를 이용한 텍스트 추출 함수
crop_and_split_image()  # 상하단 제거 + 좌우 나누기 함수


In [32]:
import os
import cv2
import io
from pdf2image import convert_from_path
from google.cloud import vision

# API 키 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/ntlkbuai-9302f377d55f.json"  # ⚠️ 파일 경로를 정확히 지정하세요

#이미지 저장 폴더 생성
os.makedirs("pages", exist_ok=True)
os.makedirs("output", exist_ok=True)

# Google Vision OCR 함수 정의
def run_ocr(image_path):
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = vision_client.document_text_detection(image=image)
    return response.full_text_annotation.text

# 이미지에서 헤더/푸터 제거 후 좌/우로 분할
def crop_and_split_image(img_path, header_ratio=0.12, footer_ratio=0.1):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    top = int(h * header_ratio)
    bottom = h - int(h * footer_ratio)
    main_body = img[top:bottom, :]

    mid = w // 2
    left = main_body[:, :mid]
    right = main_body[:, mid:]

    prefix = img_path.replace(".png", "")
    left_path = f"{prefix}_left.png"
    right_path = f"{prefix}_right.png"

    cv2.imwrite(left_path, left)
    cv2.imwrite(right_path, right)

    return left_path, right_path

# Vision API 클라이언트 초기화
vision_client = vision.ImageAnnotatorClient()

# PDF 파일을 이미지로 변환
pdf_path = "/content/2020년도 제48회 물리치료사 국가시험_1교시(홀수형).pdf"
images = convert_from_path(pdf_path, dpi=300)

# 각 페이지를 OCR 처리
for i, img in enumerate(images):
    page_num = i + 1
    img_path = f"pages/page_{page_num}.png"
    img.save(img_path)

    if page_num == 1:
        # 첫 페이지는 분할 없이 전체 OCR
        full_text = run_ocr(img_path)
    else:
        # 2페이지부터는 좌우로 나누어 OCR
        left_img, right_img = crop_and_split_image(img_path)
        text_left = run_ocr(left_img)
        text_right = run_ocr(right_img)
        full_text = f"{text_left.strip()}\n{text_right.strip()}"

    # 텍스트 파일로 저장
    with open(f"output/page_{page_num}.txt", "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"✅ 페이지 {page_num} 처리 완료")


✅ 페이지 1 처리 완료
✅ 페이지 2 처리 완료
✅ 페이지 3 처리 완료
✅ 페이지 4 처리 완료
✅ 페이지 5 처리 완료
✅ 페이지 6 처리 완료
✅ 페이지 7 처리 완료
✅ 페이지 8 처리 완료
✅ 페이지 9 처리 완료
✅ 페이지 10 처리 완료
✅ 페이지 11 처리 완료
✅ 페이지 12 처리 완료


## 🧠 3. 질문 및 보기를 블록에서 분리

이 단계에서는 각 문항 블록을 처리하여:
- 문제 본문과 보기(①~⑤ 또는 1~5)를 구분하고
- 보기 번호는 제거한 후 내용만 남깁니다.
- 최종적으로 `question_id`, `질문`, `보기가 담긴 리스트` 형식으로 저장합니다.

📌 주요 처리:
- `dap_an_start`: 정답이 시작되는 줄 번호 찾기
- 글머리 기호(`•`, `○`, `-` 등)가 있으면 줄 단위로 나누고, 없으면 이전 줄에 이어 붙임
- 정답 부분은 정규표현식으로 ①, ②, ③ 등으로 분리
- 번호(①, 1 등)는 제거 후 내용만 저장

📤 결과는 `all_questions` 리스트에 추가되며, 이후 JSON 파일로 저장됩니다.


In [40]:
import os
import re
import json

# 📁 1단계: page_1.txt부터 page_12.txt까지 내용을 하나로 합치기
folder = "/content/output"
full_text = ""

for i in range(1, 13):
    file_path = os.path.join(folder, f"page_{i}.txt")
    if os.path.exists(file_path):
        with open(file_path, encoding="utf-8") as f:
            full_text += f.read() + "\n"  # ✅ 줄바꿈과 형식을 그대로 유지함
    else:
        print(f"⚠️ 파일을 찾을 수 없습니다: {file_path}")


In [41]:
# "숫자. ..." 형식으로 시작해서 다음 숫자 앞까지 또는 EOF까지 블록으로 추출
pattern = r'(\d+\.\s.*?)(?=\n\d+\.\s|$)'
blocks = re.findall(pattern, full_text, flags=re.DOTALL)

all_questions = []  # 모든 문제를 저장할 리스트


In [42]:
for idx, block in enumerate(blocks, start=1):
    lines = block.strip().split("\n")

    # 정답이 시작되는 줄 번호 찾기
    ans_start = -1
    for i, line in enumerate(lines):
        if re.match(r'^(①|②|③|④|⑤|\d\s)', line.strip()):
            ans_start = i
            break

    if ans_start == -1 or ans_start < 1:
        print(f"⚠️ 문제 {idx}의 정답 부분을 찾을 수 없습니다.")
        continue

    # ✅ 문제 부분 정리: 줄바꿈은 유지하고, 글머리 기호는 제거
    ques_lines = lines[:ans_start]

    cleaned_lines = []
    buffer = ""

    for line in ques_lines:
        line = line.strip()

        if re.match(r'^[•\-\–●▪︎▶️★◆■☑︎✔︎➤➔❖➣➢○]+\s*', line):
            # 이전 줄이 연결 중이라면 저장
            if buffer:
                cleaned_lines.append(buffer.strip())
                buffer = ""
            # 글머리 기호 제거 후 줄 저장
            line = re.sub(r'^[•\-\–●▪︎▶️★◆■☑︎✔︎➤➔❖➣➢○]+\s*', '', line)
            cleaned_lines.append(line.strip())
        else:
            # 일반 줄이면 이전 줄에 이어 붙이기
            buffer += " " + line

    if buffer:
        cleaned_lines.append(buffer.strip())

    question = "\n".join(cleaned_lines).strip()

    # 정답 처리
    ans_lines = lines[ans_start:]
    ans_raw = "\n".join(ans_lines)

    raw_ans = re.findall(
        r'(?:①|1)\s?.*?(?=(?:②|2)\s|$)|'
        r'(?:②|2)\s?.*?(?=(?:③|3)\s|$)|'
        r'(?:③|3)\s?.*?(?=(?:④|4)\s|$)|'
        r'(?:④|4)\s?.*?(?=(?:⑤|5)\s|$)|'
        r'(?:⑤|5)\s?.*',
        ans_raw,
        flags=re.DOTALL
    )

    # 각 보기에서 번호 제거
    answer = [
        re.sub(r'^(①|②|③|④|⑤|[1-5])[\s:.\-–]*', '', d.strip())
        for d in raw_ans
    ]

    all_questions.append({
        "question_id": idx,
        "question": question,
        "answer": answer
    })


In [43]:
# 📁 JSON 파일로 저장
with open("48th_exam_full.json", "w", encoding="utf-8") as f:
    json.dump(all_questions, f, ensure_ascii=False, indent=2)

print(f"\n✅ 완료: 총 {len(all_questions)}개의 문항을 12페이지에서 추출했습니다.")



✅ 완료: 총 105개의 문항을 12페이지에서 추출했습니다.


In [46]:
import json

# 저장된 JSON 파일 읽기
with open("48th_exam_full.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 5번 문항 출력 (파이썬 인덱스는 0부터 시작하므로 data[4])
question = data[104]

print("🧾 질문:")
print(question["question"])
print("\n📌 보기:")
for i, option in enumerate(question["answer"], 1):
    print(f"{i}. {option}")


🧾 질문:
105. 다음의 환자에게 필요한 검사는?

72세 남자 환자가 뇌졸중 후 집 안에서 일상생활활동을
독립적으로 수행함 . 최근 걸을 때 흔들림으로 낙상에 대해 두려워 함

환자는 현재 팔과 다리의 기능 및 감각 상태를 확인하고 싶어 함

기능적팔뻗기 검사에서 범위가 8~12cm로 나타남

📌 보기:
1. 아프가스코어(APGAR score)
2. ESCROW 프로파일(ESCROW profile)
3. 수정된 바델 지수(modified Barthel index)
4. 퓨글-마이어 평가(Fugl-Meyer assessment)
5. 울프 운동기능검사(Wolf motor function test)
1교시 종료
